In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 500)

### 1. Подготовка

#### Данные

In [5]:
data_month=pd.read_excel('data_case.xlsx',sheet_name='month')
data_quarter=pd.read_excel('data_case.xlsx',sheet_name='quarter')
descriptions=pd.read_excel('data_case.xlsx',sheet_name='description')

#### Признаки по типам

Разделяем признаки по типам для более удобной аугментации (data augmentation)

In [6]:
# Объёмные признаки (не цены, не индексы, не доходности)
volume_features=[
    'stock_market_volume', 
    'goods_market_volume',
    'sber_volume', 
    'sber_orders',
    'spy_volume', 
    'dji_volume',
    'ixic_volume',
    'money_m2', 
    'money_m0',
    'credit_volume',
    'rkpdh_volume', 
    'vnok_volume',
    'im_volume',
    'ex_volume', 
    'gdp_volume',
    'product_taxes_volume', 
    'income_percapita_volume'
]

# Ценовые признаки
price_features=[
    'rtsi_max',
    'rtsi_min', 
    'rtsi_last', 
    'sber_last',
    'spy_close', 
    'dji_close', 
    'ixic_close',
    'brent_price',
    'real_estate_2_price', 
    'real_estate_1_price'
]

# Темпы роста (индексы)
index_features=[
    'cpi_all',
    'cpi_cars',
    'vi_retail',
    'vi_gdp',
    'pi_gdp',
    'vi_vnok',
    'vi_rkpdh'
]

# Темпы прироста (доходности)
delta_features=[
    'ofz_1', 
    'ofz_10', 
    'ofz_30'
]

# Прочие признаки
other_features=[
    'ofz_diff_10_1',
    'ofz_diff_30_10', 
    'ofz_diff_30_1',
    'unempl_rate', 
    'int_rate'
]

#### Словарь с признаками

Для каждого типа задачи (месяц/квартал) создаём свои наборы признаков

In [7]:
features_dict={
    'month':
            {
             'volume_features':[x for x in data_month.columns if x in volume_features],
             'price_features':[x for x in data_month.columns if x in price_features],
             'index_features':[x for x in data_month.columns if x in index_features],
             'delta_features':[x for x in data_month.columns if x in delta_features],
             'other_features':[x for x in data_month.columns if x in other_features]
            },
    'quarter':
            {
             'volume_features':[x for x in data_quarter.columns if x in volume_features],
             'price_features':[x for x in data_quarter.columns if x in price_features],
             'index_features':[x for x in data_quarter.columns if x in index_features],
             'delta_features':[x for x in data_quarter.columns if x in delta_features],
             'other_features':[x for x in data_quarter.columns if x in other_features]
            }
}

### 2. Генерация признаков

#### Функции для базовой генерации

In [8]:
# Абсолютный рост, темпы роста и прироста
def gen_growth(data,features):
    """
    Функция генерирует динамику признаков
    """
    data_growth=pd.DataFrame(data['period'][1:],columns=['period'])
    for c in features:
        data_growth[f'abs_growth_{c}']=data[c][1:].values-data[c][0:-1].values
        data_growth[f'index_{c}']=data[c][1:].values/data[c][0:-1].values
        data_growth[f'delta_rate_{c}']=(data[c][1:].values/data[c][0:-1].values-1)*100
        
    return data_growth

# Лаги
def gen_lag(data,features,lag):
    """
    Функция генерирует лаги признаков
    """
    data_lag=pd.DataFrame(data['period'][lag:],columns=['period'])
    for c in features:
        data_lag[f'l{lag}_{c}']=data[c][:-lag].values
        
    return data_lag

# Логарифмы
def gen_log(data,features):
    """
    Функция генерирует логарифмы признаков
    """
    data_log=pd.DataFrame(data['period'],columns=['period'])
    for c in features:
        data_log[f'log_{c}']=data[c].apply(lambda x: np.log(x) if x>0 else 0)
    
    return data_log